In [38]:
from azure.identity.aio import AzureCliCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread, Agent
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

import os
from dotenv import load_dotenv

USER_INPUTS = [
    "How can you help me?",
    "Who is Author John Grisham and why is he famous work, and what are his notable works and what's his tone like?",
    "What's the book Sooley about written by John Grisham"
]


In [39]:
api_key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment = os.getenv("DEPLOYMENT_NAME")

In [40]:
async def main():
    agent = ChatCompletionAgent(
        service=AzureChatCompletion(
            service_id="simpleAgent",
            deployment_name=os.getenv("DEPLOYMENT_NAME"),
            endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        ),
        name="Assistant",
        instructions="Answers the user's questions."
    )

    #Creates a thread
    thread: ChatHistoryAgentThread = None

    for user_input in USER_INPUTS:
        print(f"#User: {user_input}")
        response = await agent.get_response(
            messages=user_input,
            thread=thread,
        )
        print(f"# {response.name}: {response}")
        thread = response.thread
        
    await thread.delete() if thread else None

await main()

#User: How can you help me?
# Assistant: I can help you with a wide variety of tasks, including:

- **Answering questions:** I can explain concepts, provide information, and help you understand complex topics.
- **Writing assistance:** I can help you write emails, essays, reports, stories, or even poetry.
- **Proofreading and editing:** I can check your writing for grammar, spelling, and clarity.
- **Learning new skills:** I can guide you through learning new subjects, from languages to programming.
- **Brainstorming and ideas:** Need ideas for a project, gift, business, or story? I can help!
- **Problem solving:** I can assist you with logical reasoning, troubleshooting, and finding solutions.
- **Recommendations:** I can suggest books, movies, recipes, or resources based on your interests.

Let me know specifically what you need help with, and I’ll do my best to assist you!
#User: Who is Author John Grisham and why is he famous work, and what are his notable works and what's his tone

##### Multi-agent orchestration

In [41]:
def get_agents() -> list[Agent]:
    physics_agent = ChatCompletionAgent(
        name ="PhysicsAgent",
        instructions="You are an expert in physics. You answer questions from a physics perspective.",
        service=AzureChatCompletion(
            service_id="simpleAgent",
            deployment_name=os.getenv("DEPLOYMENT_NAME"),
            endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        ),
    )
    chemistry_agent = ChatCompletionAgent(
        name="ChemistrAgent",
        instructions="You are an expert in Chemistry. You answer questions from a chemitry perspective.",
        service=AzureChatCompletion(
            service_id="simpleAgent",
            deployment_name=os.getenv("DEPLOYMENT_NAME"),
            endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        ),
    )

    return [physics_agent, chemistry_agent]

In [42]:
from semantic_kernel.agents import ConcurrentOrchestration
from semantic_kernel.agents.runtime import InProcessRuntime

async def main():
    agents = get_agents()
    concurrent_orchestration = ConcurrentOrchestration(members=agents)
    runtime = InProcessRuntime()
    runtime.start()

    orchestration_result = await concurrent_orchestration.invoke(
        task="What is temperature?",
        runtime=runtime
    )

    value = await orchestration_result.get(timeout=20)
    for item in value:
        print(f"{item.name}: {item.content}")
    
    await runtime.stop_when_idle()


await main()

ChemistrAgent: **Temperature** is a physical property that quantitatively expresses the degree of "hotness" or "coldness" of a substance. From a **chemistry perspective**, temperature is fundamentally important because:

### 1. **Definition (Chemical Context)**
- Temperature measures the **average kinetic energy** of the particles (atoms, molecules, ions) in a substance.
- Higher temperature means particles move **faster**, and lower temperature means they move **slower**.

### 2. **Role in Chemical Reactions**
- Temperature affects the **rate of chemical reactions**. As temperature increases, particles have more energy, collide more frequently, and reactions generally proceed faster.
- It can also influence **equilibrium positions** and **phase changes** (melting, boiling, sublimation).

### 3. **Units**
- Common units are **degrees Celsius (°C)**, **Kelvin (K)** (SI unit), and sometimes **degrees Fahrenheit (°F)**.

### 4. **Measurement**
- Temperature is measured using thermometers,